# Chapitre 3: Traitement des données

 
## Changement des types de variables

In [ ]:
alltrain.timestamp=pd.to_datetime(alltrain['timestamp'])

In [ ]:
varia=['building_id','site_id', 'meter']

for col in varia:
    
    alltrain[col]=alltrain[col].astype('object')
    

In [ ]:
alltrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 16 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         object        
 1   meter               object        
 2   timestamp           datetime64[ns]
 3   meter_reading       float64       
 4   site_id             object        
 5   primary_use         object        
 6   square_feet         int64         
 7   year_built          float64       
 8   floor_count         float64       
 9   air_temperature     float64       
 10  cloud_coverage      float64       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float64       
 13  sea_level_pressure  float64       
 14  wind_direction      float64       
 15  wind_speed          float64       
dtypes: datetime64[ns](1), float64(10), int64(1), object(4)
memory usage: 2.6+ GB


In [ ]:
alltrain['meter'] = pd.Categorical(alltrain['meter']).rename_categories({0: 'electricity', 
                                                                   1: 'chilledwater',
                                                                   2: 'steam', 
                                                                   3: 'hotwater'})

## Occurence des modalités de chaque variable quali

In [ ]:
 for col in alltrain.select_dtypes(object).columns:
    print (f'{col :-<30} {len(alltrain[col].value_counts())}')

building_id------------------- 1449
site_id----------------------- 16
primary_use------------------- 16


## Analyse des valeurs manquantes

In [ ]:
(alltrain.isna().sum()/alltrain.shape[0]).sort_values(ascending=False)

floor_count           0.826528
year_built            0.599900
cloud_coverage        0.436551
precip_depth_1_hr     0.185447
wind_direction        0.071678
sea_level_pressure    0.060925
wind_speed            0.007107
dew_temperature       0.004953
air_temperature       0.004781
square_feet           0.000000
primary_use           0.000000
site_id               0.000000
meter_reading         0.000000
timestamp             0.000000
meter                 0.000000
building_id           0.000000
dtype: float64

In [ ]:
alltrain.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,electricity,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,electricity,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,electricity,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,electricity,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,electricity,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


## Extraction de features

In [ ]:
def rec_heur(x):
    
    if x in np.arange(6, 19):
        return 'journee'
    
    if x in np.arange(19, 23):
        return 'nuit'
    
    if x in [23, 0, 1, 2, 3, 4, 5]:
        return 'tard'

In [ ]:
alltrain['year_built'].describe()

count    8.088455e+06
mean     1.968277e+03
std      3.020815e+01
min      1.900000e+03
25%      1.951000e+03
50%      1.969000e+03
75%      1.993000e+03
max      2.017000e+03
Name: year_built, dtype: float64

In [ ]:
def discredit_var(x):
   
    
    if x <= 1951:
        return 'yearB_q1'
    
    if  1951 < x <= 1969:
        return 'yearB_q2'
    
    if 1969 < x <= 1993:
        return 'yearB_q3'
    
    if  1993 < x:
        return 'yearB_q4'
    

In [ ]:
def preProcecing_df(df_):
    
    df=df_.copy()
    reduce_mem_usage(df)
    
    
    saison={3: 'printent',4:'printent',5:'printent',
          6: 'ete', 7: 'ete',8: 'ete', 
          9: 'automne', 10: 'automne', 11: 'automne', 
          1: 'hiver', 12: 'hiver', 2: 'hiver'}
      
    
    df['mois'] = df.timestamp.dt.month
    df['day'] = df.timestamp.dt.day
    df['heure'] = df.timestamp.dt.hour
    reduce_mem_usage(df)
    
    df['heureDiscredite'] = df['heure'].apply(rec_heur)
    reduce_mem_usage(df)
    
    df['week_end'] = [1 if x in [5,6] else 0 for x in df.day]
    reduce_mem_usage(df)
    df['saison'] = df['mois'].apply(lambda x: saison.get(x))
    reduce_mem_usage(df)
    median_group = df.groupby(['site_id'])['year_built'].transform('median')
    reduce_mem_usage(df)
    df['year_built'].fillna(median_group,inplace = True)
    reduce_mem_usage(df)
    df['year_built'].fillna(df['year_built'].median(), inplace=True)
    reduce_mem_usage(df)
    df['year_built'] = df['year_built'].apply(discredit_var) 
    reduce_mem_usage(df)
    df.floor_count.fillna(0,inplace = True)
    reduce_mem_usage(df)
    colonneAsNum=['air_temperature', 'dew_temperature','wind_direction']
    for col in colonneAsNum:
        median_group = df.groupby(['site_id', 'saison', 'week_end', 'primary_use'])[col].transform('median')
        df[col].fillna(median_group,inplace = True)
    reduce_mem_usage(df)                     
    for col in [ 'day', 'heure', 'timestamp', 
                "precip_depth_1_hr", "wind_speed", "sea_level_pressure", "cloud_coverage", "mois"]:
        del df[col]
    reduce_mem_usage(df)
    
    return df



In [ ]:
X = preProcecing_df(alltrain)


Memory usage after optimization is: 1041.10 MB
Decreased by 0.0%
Memory usage after optimization is: 1098.94 MB
Decreased by 26.9%
Memory usage after optimization is: 1253.17 MB
Decreased by 0.0%
Memory usage after optimization is: 1272.45 MB
Decreased by 9.6%
Memory usage after optimization is: 1426.69 MB
Decreased by 0.0%
Memory usage after optimization is: 1426.69 MB
Decreased by 0.0%
Memory usage after optimization is: 1426.69 MB
Decreased by 0.0%
Memory usage after optimization is: 1426.69 MB
Decreased by 0.0%
Memory usage after optimization is: 1542.37 MB
Decreased by 0.0%
Memory usage after optimization is: 1542.37 MB
Decreased by 0.0%
Memory usage after optimization is: 1542.37 MB
Decreased by 0.0%
Memory usage after optimization is: 1176.05 MB
Decreased by 0.0%


In [ ]:
X.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,dew_temperature,wind_direction,heureDiscredite,week_end,saison
0,0,electricity,0.0,0,Education,7432,yearB_q4,0.0,25.0,20.0,0.0,tard,0,hiver
1,1,electricity,0.0,0,Education,2720,yearB_q4,0.0,25.0,20.0,0.0,tard,0,hiver
2,2,electricity,0.0,0,Education,5376,yearB_q3,0.0,25.0,20.0,0.0,tard,0,hiver
3,3,electricity,0.0,0,Education,23685,yearB_q4,0.0,25.0,20.0,0.0,tard,0,hiver
4,4,electricity,0.0,0,Education,116607,yearB_q3,0.0,25.0,20.0,0.0,tard,0,hiver


In [ ]:
X1=X.copy()

In [ ]:
reduce_mem_usage(X1)

In [ ]:
X1.loc[  (X1.site_id==0)&(X1.meter == 'electricity') , 'meter_reading'] = X1.loc[(X1.site_id==0) & (X1.meter == 'electricity') , 'meter_reading'] * 0.2931

In [ ]:
X =  X[X['meter_reading']!= np.float(0)]
X['meter_reading']=np.log1p(X['meter_reading'])

In [ ]:
(X.isna().sum()/X.shape[0]).sort_values(ascending=False)

saison             0.0
week_end           0.0
heureDiscredite    0.0
wind_direction     0.0
dew_temperature    0.0
air_temperature    0.0
floor_count        0.0
year_built         0.0
square_feet        0.0
primary_use        0.0
site_id            0.0
meter_reading      0.0
meter              0.0
building_id        0.0
dtype: float64

In [ ]:
X.shape

(18342124, 14)

In [ ]:
X.to_csv (r'/content/X.csv', index = False, header=True)

## conversion type

In [ ]:
X["building_id"] = X["building_id"].astype('category')
X["site_id"] = X["site_id"].astype('category')
X["primary_use"] = X["primary_use"].astype('category')
X["saison"] = X["saison"].astype('category')
X["heureDiscredite"] = X["heureDiscredite"].astype('category')
X["year_built"] = X["year_built"].astype('category')

## Encodeur One Hot

In [ ]:
def encodeur(df): 
    X_Encod=pd.concat([df, pd.get_dummies(df["primary_use"], dtype=int) ], axis=1)
    reduce_mem_usage(df)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["saison"], dtype=int) ], axis=1)
    reduce_mem_usage(df)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["heureDiscredite"], dtype=int) ], axis=1)
    reduce_mem_usage(df)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["meter"], dtype=int) ], axis=1)
    reduce_mem_usage(df)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["year_built"], dtype=int) ], axis=1)
    reduce_mem_usage(df)

    for col in ["primary_use",'year_built', 'yearB_q4', "saison", "heureDiscredite", 
                'Office', "printent", "journee", 'meter', 'hotwater' ]:
        del X_Encod[col]


    return X_Encod

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
X_Encod = pd.read_csv('/content/drive/MyDrive/Kaggle/X_Encod.csv')

In [ ]:
X_Encod = encodeur(X)

In [ ]:
X_Encod.info()

In [ ]:
X_Encod.to_csv (r'/content/drive/MyDrive/Kaggle/X_Encod.csv', index = False, header=True)

In [ ]:
reduce_mem_usage(X_Encod)

## Train test split

In [ ]:
from sklearn.model_selection import ShuffleSplit

def trainAndTest(DF):
    df= DF.copy()
    
    uniqueSite=list(pd.unique(df["site_id"]))
    rs = ShuffleSplit(n_splits=1, test_size=.3, random_state=0)
    for train_index, test_index in rs.split(uniqueSite):

        df['trainIndex'] = [1 if x in train_index else 0 for x in df.site_id]
        x_train = df[df['trainIndex']==1]
        y_train = x_train['meter_reading']

        x_test = df[df['trainIndex']==0]
        y_test = x_test['meter_reading']

    del x_train['trainIndex'] 
    del x_train['meter_reading'] 

    del x_test['trainIndex']
    del x_test['meter_reading']
    
    return x_train, y_train, x_test, y_test


In [ ]:
 X_train, Y_train, X_test, Y_test = trainAndTest(X_Encod)

In [ ]:
X_train=reduce_mem_usage(X_train)
X_test=reduce_mem_usage(X_test)

## MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
def minMax(DF, listColumns, scaler):
    df=DF.copy()
    for col in listColumns:
        df[col]=scaler.fit_transform(df[[col]])
    reduce_mem_usage(df)  
    return df

In [ ]:
listColumns= [ 'wind_direction',  'dew_temperature',
              'air_temperature', 'floor_count', 'square_feet']


X_train = minMax(X_train, listColumns, scaler)
X_test = minMax(X_test, listColumns, scaler)